In [1]:
import torch
import torch.nn as nn
from torchvision import datasets ,models,transforms
from torch.utils.data.sampler import SubsetRandomSampler
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np

In [2]:
dataPath = Path("./dvsc/train")
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder(dataPath, transform=train_transforms)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self,activation='sigmoid'):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        if activation == 'sigmoid':
            self.ann = nn.Sequential(
                nn.Linear(224*224*3, 1024),
                nn.Sigmoid(),
                nn.Linear(1024, 512),
                nn.Sigmoid(),
                nn.Linear(512, 1),
                nn.Sigmoid()
            )
        if activation == 'tanh':
            self.ann = nn.Sequential(
                nn.Linear(224*224*3, 1024),
                nn.Tanh(),
                nn.Linear(1024, 512),
                nn.Tanh(),
                nn.Linear(512, 1),
                nn.Sigmoid()
            )
        if activation == 'linear':
            self.ann = nn.Sequential(
                nn.Linear(224*224*3, 1024),
                nn.Linear(1024,1024),
                nn.Linear(1024, 512),
                nn.Linear(512,512),
                nn.Linear(512, 1),
                nn.Sigmoid()
            )   
        
    def forward(self,x):
        x = self.flatten(x)
        return self.ann(x)

def reset_weights(m):
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()


In [4]:
linear_model = NeuralNetwork(activation='linear')
sigmoid_model = NeuralNetwork(activation='sigmoid')
tanh_model = NeuralNetwork(activation='tanh')

In [5]:
print("linear_model: \n",linear_model)
print("sigmoid_model: \n",sigmoid_model)
print("tanh_model: \n",tanh_model)

linear_model: 
 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (ann): Sequential(
    (0): Linear(in_features=150528, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=1024, bias=True)
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)
sigmoid_model: 
 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (ann): Sequential(
    (0): Linear(in_features=150528, out_features=1024, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)
tanh_model: 
 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (ann): Sequential(
    (0): Linear(in_features=150528, out_features=1024, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1024,

In [6]:
from sklearn.model_selection import KFold
k_folds = 5 
num_epochs = 3
loss_function = nn.BCELoss()

results = {} 
torch.manual_seed(22)
kfold = KFold(n_splits = k_folds,shuffle=True)
print("Initiate kfold")
print(len(train_data))

Initiate kfold
400


In [7]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

for fold, (train_ids,test_ids) in enumerate(kfold.split(train_data)):
    # Print
    print(f'FOLD {fold}')
    print('+-+-+-+-+-+-+-+-+-+')
    
        # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      train_data, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      train_data,
                      batch_size=10, sampler=test_subsampler)
    print(len(trainloader))
    print(len(testloader))
    # Init the neural network
    linear_model.apply(reset_weights)
    sigmoid_model.apply(reset_weights)
    tanh_model.apply(reset_weights)
    
    #initialize optimizer
    optimizer_linear = torch.optim.Adam(linear_model.parameters(), lr=1e-3)
    optimizer_sigmoid = torch.optim.Adam(sigmoid_model.parameters(), lr=1e-3)
    optimizer_tanh = torch.optim.Adam(tanh_model.parameters(), lr=1e-3)
    
    
    
    #RUN THE TRAINING LOOP 
    for epoch in range(0,num_epochs):
        print(f'Starting epoch {epoch+1}')

        current_loss_linear = 0.0
        current_loss_sigmoid = 0.0
        current_loss_tanh = 0.0
    
        #Iterate over the dataloader 
        for i,data in enumerate(trainloader,0):
            inputs, targets = data 

            optimizer_linear.zero_grad()
            optimizer_sigmoid.zero_grad()
            optimizer_tanh.zero_grad()

            outputs_linear = linear_model(inputs).flatten()
            outputs_sigmoid = sigmoid_model(inputs).flatten()
            outputs_tanh = tanh_model(inputs).flatten()

            loss_linear = loss_function(outputs_linear,targets.float())
            loss_sigmoid = loss_function(outputs_sigmoid,targets.float())
            loss_tanh = loss_function(outputs_tanh,targets.float())

            loss_linear.backward()
            loss_sigmoid.backward()
            loss_tanh.backward()

            optimizer_linear.step()
            optimizer_sigmoid.step()
            optimizer_tanh.step()

            current_loss_linear += loss_linear.item()
            current_loss_sigmoid += loss_sigmoid.item()
            current_loss_tanh += loss_tanh.item()
            if i % 20 == 19:
                print('Linear Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss_linear / 20))
                print('Sigmoid Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss_sigmoid / 20))
                print('Tanh Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss_tanh / 20))
                current_loss_linear = 0.0
                current_loss_sigmoid = 0.0
                current_loss_tanh = 0.0

    print('Training process has finished.')

    print('Starting testing')
        
    predicted_values_linear = []
    predicted_values_sigmoid = []
    predicted_values_tanh = [] 
    
    true_values = [] 
    with torch.no_grad():
        for i,data in enumerate(testloader,0):
            inputs,targets = data

            outputs_linear = linear_model(inputs)
            outputs_sigmoid = sigmoid_model(inputs)
            outputs_tanh = tanh_model(inputs)
            
            outputs_linear = (outputs_linear>0.5)
            outputs_sigmoid = (outputs_sigmoid>0.5)
            outputs_tanh = (outputs_tanh>0.5)
            
            predicted_values_linear = np.append(predicted_values_linear,outputs_linear)
            predicted_values_sigmoid = np.append(predicted_values_sigmoid,outputs_sigmoid)
            predicted_values_tanh = np.append(predicted_values_tanh,outputs_tanh)
            
            true_values= np.append(true_values,targets)

        val_confusion_matrix_linear = confusion_matrix(predicted_values_linear,true_values)
        val_precision_score_linear = precision_score(predicted_values_linear,true_values,average='weighted')
        val_recall_score_linear = recall_score(predicted_values_linear,true_values,average='weighted')
        val_f1_score_linear = f1_score(predicted_values_linear,true_values,average='weighted')

        val_confusion_matrix_sigmoid = confusion_matrix(predicted_values_sigmoid,true_values)
        val_precision_score_sigmoid = precision_score(predicted_values_sigmoid,true_values,average='weighted')
        val_recall_score_sigmoid = recall_score(predicted_values_sigmoid,true_values,average='weighted')
        val_f1_score_sigmoid = f1_score(predicted_values_sigmoid,true_values,average='weighted')

        val_confusion_matrix_tanh = confusion_matrix(predicted_values_tanh,true_values)
        val_precision_score_tanh = precision_score(predicted_values_tanh,true_values,average='weighted')
        val_recall_score_tanh = recall_score(predicted_values_tanh,true_values,average='weighted')
        val_f1_score_tanh = f1_score(predicted_values_tanh,true_values,average='weighted')

    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_')
    print("Linear Model Evaluation")
    print(f' Linear Model Precision Score: {val_precision_score_linear }')
    print(f' Linear Model Recall Score: {val_recall_score_linear}')
    print(f' Linear Model F1 Score: {val_f1_score_linear}')
    print(f' Linear Model Confusion Matrix: \n{val_confusion_matrix_linear}')
    print("****************************************************")
    
    print("Sigmoid Model Evaluation")
    print(f' Sigmoid Model Precision Score: {val_precision_score_sigmoid }')
    print(f' Sigmoid Model Recall Score: {val_recall_score_sigmoid}')
    print(f' Sigmoid Model F1 Score: {val_f1_score_sigmoid}')
    print(f' Sigmoid Model Confusion Matrix: \n{val_confusion_matrix_sigmoid}')
    print("****************************************************")
    
    print("Tanh Model Evaluation")
    print(f' Tanh Model Precision Score: {val_precision_score_tanh }')
    print(f' Tanh Model Recall Score: {val_recall_score_tanh}')
    print(f' Tanh Model F1 Score: {val_f1_score_tanh}')
    print(f' Tanh Model Confusion Matrix: \n{val_confusion_matrix_tanh}')
    print("****************************************************")

FOLD 0
+-+-+-+-+-+-+-+-+-+
32
8
Reset trainable parameters of layer = Linear(in_features=150528, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=512, bias=True)
Reset trainable parameters of layer = Linear(in_features=512, out_features=512, bias=True)
Reset trainable parameters of layer = Linear(in_features=512, out_features=1, bias=True)
Reset trainable parameters of layer = Linear(in_features=150528, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=512, bias=True)
Reset trainable parameters of layer = Linear(in_features=512, out_features=1, bias=True)
Reset trainable parameters of layer = Linear(in_features=150528, out_features=1024, bias=True)
Reset trainable parameters of layer = Linear(in_features=1024, out_features=512, bias=True)
Reset trainable parameters of layer = Linear(

Looking at the F1 score, we see that sigmoid activation performs well for classification task. However, we have not tested the model with different learning rate and different model with regularization. Further testing could give more insights on the data. 